In [12]:
import pydicom
import pickle
from sklearn.svm import SVC
import numpy as np
import zipfile
from scipy.special import expit
import pandas as pd
from sklearn.utils import shuffle

In [2]:
class Encoder:
    def __init__(self, model):
        self.n_comp = model.n_comp
        self.l1 = model.encoder[0].weight.data.numpy()
        self.l1b = model.encoder[0].bias.data.numpy()

        self.l2 = model.encoder[2].weight.data.numpy()
        self.l2b = model.encoder[2].bias.data.numpy()

        self.l3 = model.encoder[4].weight.data.numpy()
        self.l3b = model.encoder[4].bias.data.numpy()

    def encode(self, X):
        X = X.dot(self.l1.T) + self.l1b
        X = expit(X)
        X = X.dot(self.l2.T) + self.l2b
        X = expit(X)
        X = X.dot(self.l3.T) + self.l3b

        #sparce
        z = X
        z = np.sort(X, axis=1)[:,::-1][:, self.n_comp - 1]#.resize((len(x), 1))
        z = np.array([z])
        z.resize((len(X), 1))
        X[X < z] = 0
        #delete 0
        X[X != 0].resize((len(X), self.n_comp))
        k = len(X)
        z = np.array([X[X != 0]])
        z.resize((k, self.n_comp))

        return z

In [3]:
with open('../data/other.pickle', 'rb') as f:
    other = pickle.load(f)

In [4]:
with open('../data/d2data64.pickle', 'rb') as f:
    d2data64 = pickle.load(f)

In [5]:
with open('../data/y.pickle', 'rb') as f:
    y = pickle.load(f)

In [6]:
with open('../app/src/model/encoder.pickle', 'rb') as f:
    ae = pickle.load(f)

In [13]:
data = ae.encode(d2data64)
data = pd.DataFrame(data)
data = pd.concat([data, other], axis=1)
data, y_ = shuffle(data, y)
res_svc = SVC(kernel='rbf', shrinking=True, probability=False, degree=3, gamma='auto')
res_svc.fit(data, y_)
res_svc

SVC(gamma='auto')

In [14]:
with open('../app/src/model/predictor-1.1.1.pickle', 'wb') as f:
    pickle.dump(res_svc, f)